In [1]:
import os
import json
import copy
import numpy as np
import pandas as pd
from pathlib import Path
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *
from datatools.maneger import DataManager

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
model_path = "../models/context/"
model_name = "topic_ada.pickle"
modelM = DataManager(model_path)
print(model_name)

topic_ada.pickle


In [3]:
from sklearn.ensemble import AdaBoostClassifier
clf = modelM.load_data(model_name)

success load : ../models/context/topic_ada.pickle


In [4]:
errors = ["Topic transition error", "Unclear intention", "Lack of information"]
# errors = ["Lack of information"]
errors = ["Topic transition error"]
# errors = ["Unclear intention"]
length = 2

In [7]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']

output = "./"

def read_json_with_NoErr(path:str, datalist:list) -> pd.DataFrame:
    cols = ['did', 'tid', 'usr', 'sys', 'ec']
    df = pd.DataFrame(index=[], columns=cols)

    for p in datalist:
        datapath = Path(path + p + '/')
        for file in datapath.glob("*.json"):
            with open(file, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    if t["turn-index"] == 0:
                        continue
                    if t["speaker"] == "U":
                        usr = t["utterance"]
                        continue
                    if t["speaker"] == "S" :
                        tid = t["turn-index"]
                        sys = t["utterance"]
                        if t["error_category"]:
                            ec = t["error_category"]
                        else:
                            ec = ["No-Err"]
                        df = df.append(pd.DataFrame([did, tid, usr, sys, ec], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [8]:
df = read_json_with_NoErr(path, datalist)

In [9]:
leng_c = 2
def make_X_str_y(df, errors):
    X_str = []
    y = []
    y = np.zeros(len(df))
    continue_conv = []
    did = 0
    for i, (d, u, s, e) in enumerate(zip(df.did, df.usr, df.sys, df.ec)):
        # sentence_vectors = Nmodel.encode([u, s])
        if d != did:
            continue_conv = [""]*leng_c
            did = d
        continue_conv.append(u)
        continue_conv.append(s)
        X_str.append( continue_conv[-leng_c:] )
        for err in errors:
            if err in e:
                y[i] = 1

    return X_str, y

In [11]:
X_str, y = make_X_str_y(df, errors)

In [16]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
model_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [17]:
def make_X(convs):
    # emb_dim = nlp("形態素").vector.shape
    X_data = []
    
    for conv in convs :
        sentence_vectors = Nmodel.encode(conv)
        X_data.append(sentence_vectors[1]- sentence_vectors[0])
    return np.array(X_data)

In [18]:
X = make_X(X_str)

In [20]:
y_pred = clf.predict(X)

In [21]:
y_pred

array([1., 1., 0., ..., 0., 0., 1.])

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('F-measure: ', metrics.f1_score(y, y_pred))

confusion matrix = 
 [[1658  150]
 [ 167   25]]
accuracy =  0.8415
F-measure:  0.1362397820163488


In [28]:
correct_n = 0
bad_n = 0
all_tp_one = 0

PF = 0
FP = 0
for x, t, n in zip(X_str, y, y_pred):
    # t==1 : 本来の破綻
    # n==1 : 予想された破綻
    if t == 1:
        all_tp_one += 1
        if n == 1:
            # t==n==1 : 適切に検出
            correct_n += 1
            # print(x)
        else:
            # 破綻なのに未検出
            bad_n += 1
            FP += 1
            # print(x)
    else:
        # 破綻ではないのに破綻扱い
        if n == 1:
            bad_n += 1
            PF += 1
            print(x)
print("corrent_n:", correct_n)
print("all_tp_one:", all_tp_one)
print("bad_n, PF, FP:", bad_n, PF, FP)
print("rate c:", correct_n/all_tp_one)

['そうですね。最近とても暑いですから。', '海に行かないんですね？']
['休日に行きたいと思います。', '海は大丈夫ですね']
['ダイエット中の人は、なるべく野菜中心の食事にするとよさそうですね。', 'カロリーは欲しいですね']
['東京もおすすめです', '温泉を楽しんでるんです']
['毎日知りたいとは思いませんね。', '確かに。。']
['この付近に美味しいお店があるかな？', '付近は紅葉が進むようです']
['科学がお好きですか？', 'ご想像にお任せします。']
['今日はいい天気ですね。', 'どこか出かけるんですか？']
['興味があるスポーツは何ですか？', '今は、してないですね。']
['学生時代に何かスポーツしていましたか？', '今は、してないですね。']
['気持ちいいよね', '海は好きですね']
['楽しいでしょ？', '泳ぐを覚えるのです']
['どうやって覚えたの？', '泳ぐを繰り返すようですよ']
['今日は雨が降っていますね。', '雨は大丈夫ですね']
['マッサージ屋行きたい', '腰をいたわりますよねえ']
['うん。けっこう会えるよ。', 'エゾシカが姿を見せるのです']
['こんにちは。暑いですね。', 'ところで、気になるニュースがあったのですが、7月29日から公開された映画「シン・ゴジラ」の観客数が1日までに71万人に達し、1954年に第1作が公開された「ゴジラ」シリーズの累計観客数が1億人突破し、実写映画では初めてだそうですよ。']
['うん。そういうこと多いです。', '素通りを許されてないな']
['なんで思考力や計算力もスポーツなんですか？', '今は、してないですね。']
['ああ。周りの女子の話か。そう？', '予防を怠るようです']
['勝っちゃたんだ！つまり君は僕よりもどきどきしてるという意味？', '予防を心掛けますよねえ']
['なんの種目ですか？', 'メダルが決まります']
['ペトリコールって初めて聞いたよ。科学に詳しいんだね。', '科学の進歩は、目を見張るものがあるよね。']
['とても面白いよ', 'ビーチバレーで有名ですね']
['有名な人がいる？', 'ビーチバレーは熱いですよねえ']
['当たらないと思います', '女の人ほど、血液型にこだわるよね。']
['